In [ ]:
# training:
#   root_dir: artifacts/training
#   trained_model_path: artifacts/training/model.h5

In [2]:
import os
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition'

In [4]:
#Entity 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [5]:
from Sign_Language.constants import *
from Sign_Language.utils.common import create_directories, read_yaml

In [6]:
# Configuration Manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])


        get_prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )

        return get_prepare_callbacks_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params.baseModelParams

        training_data = Path(self.config.data_ingestion.train_data_path)

        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            base_model_path = Path(prepare_base_model.base_model_path),
            training_data = training_data,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            )
        
        return training_config

In [12]:
import time
import os
import tensorflow as tf
import pandas as pd
import numpy as np

In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [20]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.base_model_path)
    
    def train_valid_generator(self):

        data = pd.read_csv(self.config.training_data, index_col = "Unnamed: 0")
        self.X = np.array(data.drop('label', axis = 1))
        self.y = np.array(data[['label']])
        
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        
        self.model.fit(self.X, self.y, batch_size=self.config.params_batch_size, 
                       epochs=self.config.params_epochs, validation_split=0.2, 
                       callbacks=callback_list)
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [21]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-02-09 02:24:04,178: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-09 02:24:04,186: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-09 02:24:04,190: INFO: common: created directory at: artifacts]
[2024-02-09 02:24:04,193: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-02-09 02:24:04,196: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-02-09 02:24:04,199: INFO: common: created directory at: artifacts\training]


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.